In [ ]:
import pandas as pd

df = pd.read_excel('cities_streetview_run_1.xlsx', engine='openpyxl')

# Filter the rows based on the date condition
print("CITIES STREETVIEWED: ")
print("within the last year: ",len(df[df['date'] >= '2022-11']))
print("within the last 6 months: ",len(df[df['date'] >= '2023-05']))
print("within the past 3 months: ",len(df[df['date'] >= '2023-08']))
print("within the past month: ",len(df[df['date'] >= '2023-10']))



In [16]:
## GRAB SINGLE IMAGE, AND PASS IT TO THE MODEL
from sign_url import sign_url
from dotenv import load_dotenv
import requests
import os

URL_SIGNING_SECRET = os.getenv("URL_SIGNING_SECRET")
GOOGLE_MAPS_API_KEY = os.getenv("GOOGLE_MAPS_API_KEY")

# Sample is london, Ontario
# {'copyright': '© Google', 'date': '2023-07', 'location': {'lat': 42.98362377713629, 'lng': -81.24965439993474}, 'pano_id': 'r-501hayIC6PKLB9K1NT9g', 'status': 'OK'}
parameters = {
    "location": '42.9836, -81.2497',
    "size": '640x640',
    "return_error_code": 'true',
    "key": GOOGLE_MAPS_API_KEY,
    "fov": 120,
    "heading": 0,
    "pitch": 0,
    "source": "outdoor",
    "radius": 50
}
p_string = [f"{key}={value}&" for key, value in parameters.items()]
p_string[-1] = '' # remove last &

unsigned_url = f"https://maps.googleapis.com/maps/api/streetview?{p_string}"
signed_url = sign_url(unsigned_url, URL_SIGNING_SECRET)

# pano_id are dynamic, use location instead
response = requests.get(signed_url, stream=True)

file_path = "../../images/sample_image.jpg"
if response.status_code == 200:
    with open(file_path, 'wb') as file:
        file.write(response.content)
        
    size_in_bytes = os.path.getsize(file_path)
    size_in_kilobytes = size_in_bytes / 1024  # Convert size to KB for easier readability
    print(f"Image saved as {file_path} with size {size_in_kilobytes:.2f} KB")
else:
    print("Error fetching image:", response.status_code)

Image saved as ../../images/sample_image.jpg with size 76.71 KB


In [ ]:
import pandas as pd

load_dotenv()
GOOGLE_MAPS_API_KEY = os.getenv('GOOGLE_MAPS_API_KEY')
URL_SIGNING_SECRET = os.getenv('URL_SIGNING_SECRET')

full_df = pd.read_excel('north-america-cities.csv')
filtered_df = pd.read_excel('cities_streetview_full_run.xlsx', engine='openpyxl')

# Takes city and location, and returns list of unique coordinates where streetview images were taken
def streetview_gridsearch(city, lat, lng):
    DEGREE_IN_METERS = 111139
    diff = 50/DEGREE_IN_METERS
    
    # Calculate using the formula: length = (population / density) ** 0.5
    city_length = (full_df[city]["population"] / full_df[city]["density"]) ** 0.5
    locations = []
    
    # Search in a square of 110% of the city's theoretical length
    step_length = 0.05 # 50 meters
    grid_length = (city_length * 1.1) / step_length
    for x in range(grid_length):
        for y in range(grid_length):
            locations.append((lat + (x * step_length) - (grid_length * step_length / 2), lng + (y * step_length) - (grid_length * step_length / 2)))

    for location in locations:
        unsigned_url = f"https://maps.googleapis.com/maps/api/streetview/metadata?location={location[0]},{location[1]}&key={GOOGLE_MAPS_API_KEY}"
        signed_url = sign_url(unsigned_url, URL_SIGNING_SECRET)

        response = requests.get(signed_url, stream=True).json()
        # print("response: ", response)
        if response["status"] == "OK":
            return {"city": city_name,
                    "date": response["date"],
                    "lat": response["location"]["lat"],
                    "lng": response["location"]["lng"],
                    "status": "OK"}
        elif response["status"] == "ZERO_RESULTS":
            continue
        else:
            raise Exception("Error: ", response["status"])

    return {"city": city_name,
            "date": None,
            "lat": None,
            "lng": None,
            "status": "NOT_FOUND"}